In [1]:
!pip install opencv-python google-cloud-vision


  Using cached google_cloud_vision-3.10.1-py3-none-any.whl.metadata (9.5 kB)
  Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_auth-2.39.0-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached grpcio-1.72.0rc1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached grpcio_status-1.72.0rc1-py3-none-any.whl.metadata (1.1 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.man

In [10]:
!export GOOGLE_APPLICATION_CREDENTIALS="nimble-root-457808-r2-b639a6729402.json"

In [11]:
import cv2
import io
from google.cloud import vision

In [12]:
def main():
    # Inicializa el cliente de Vision API
    client = vision.ImageAnnotatorClient()

    # Abre la webcam (índice 0)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: No se pudo abrir la webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Codifica el frame como JPEG en memoria
        ret2, buf = cv2.imencode('.jpg', frame)
        if not ret2:
            continue
        content = buf.tobytes()

        # Crea la imagen para enviar a la API
        image = vision.Image(content=content)

        # Llama a la detección de objetos
        response = client.object_localization(image=image)
        objects = response.localized_object_annotations

        # Dimensiones del frame
        h, w, _ = frame.shape

        # Dibuja bounding boxes para cada persona detectada
        for obj in objects:
            if obj.name.lower() == 'person':
                verts = obj.bounding_poly.normalized_vertices
                # Convierte coordenadas normalizadas a píxeles
                pts = [(int(v.x * w), int(v.y * h)) for v in verts]
                if pts:
                    x_coords = [p[0] for p in pts]
                    y_coords = [p[1] for p in pts]
                    x_min, x_max = min(x_coords), max(x_coords)
                    y_min, y_max = min(y_coords), max(y_coords)
                    # Dibuja el rectángulo y etiqueta
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    cv2.putText(frame, 'Person', (x_min, y_min - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Muestra el frame resultante
        cv2.imshow('Detección de Personas', frame)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [13]:
if __name__ == '__main__':
    main()

Error: No se pudo abrir la webcam.


[ WARN:0@63.154] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@63.154] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
